In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Configuration du navigateur
chrome_options = Options()
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")


driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()


data = []

try:
    
    url = 'https://learn.microsoft.com/fr-fr/credentials/certifications/azure-ai-fundamentals/?practice-assessment-type=certification'
    driver.get(url)
    time.sleep(5)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'article')))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    article = soup.find('article', class_='card border-color-accent border-left-lg card-horizontal')

    if not article:
        raise Exception("Article non trouvé")

    # Récupérer les liens (ignorer les 2 premiers)
    liens = article.find_all('a')[2:]
    for lien in liens:
        module_name = lien.get_text(strip=True)
        href = lien.get('href')
        if not href.startswith("http"):
            href = "https://learn.microsoft.com" + href

        print(f"\n--- MODULE : {module_name} ---\nURL : {href}")
        driver.get(href)
        time.sleep(4)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        time.sleep(1)



        soup = BeautifulSoup(driver.page_source, 'html.parser')
        units_cards = soup.find_all('div', class_='padding-sm padding-bottom-none-tablet')
        for card in units_cards:
            print(f"\n--- CARD ---")
            button = soup.find_all('button', class_='unit-expander')
            buttons = driver.find_elements(By.XPATH, "//button[.//span[text()='Vue d’ensemble']]")
            for i, button in enumerate(buttons):
                time.sleep(3)
                try:
                    aria_expanded = button.get_attribute("aria-expanded")
                    if aria_expanded == "false":
                        print(f" ➤ Ouverture du bouton Vue d’ensemble #{i+1}")
                        driver.execute_script("arguments[0].click();", button)
            
                        
                except Exception as e:
                    print(f"   ⚠️ Erreur lors du clic sur le bouton Vue d’ensemble #{i+1}: {str(e)}")
                
            updated_html = driver.page_source
            updated_soup = BeautifulSoup(updated_html, 'html.parser')

            unit_links = updated_soup.find_all('a', class_='unit-title')
            for unit_link in unit_links:
                unit_name = unit_link.get_text(strip=True)
                unit_href = unit_link['href']
                if not unit_href.startswith("http"):
                    unit_href = "https://learn.microsoft.com" + unit_href

                driver.get(unit_href)
                try:
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'module-unit-content')))
                    unit_soup = BeautifulSoup(driver.page_source, 'html.parser')
                    content_div = unit_soup.find('div', id='module-unit-content')
                    content_text = content_div.get_text(separator="\n", strip=True) if content_div else ""

                    data.append({
                        'module_name': module_name if 'module_name' in locals() else '',  # Avoid NameError
                        'unit_name': unit_name,
                        'content': content_text,
                        'source_url': unit_href
                    })

                except Exception as e:
                    print(f"Erreur sur l'unité '{unit_name}' ({unit_href}) : {str(e)}")

finally:
    driver.quit()

# Création du DataFrame
df = pd.DataFrame(data)
print(df.head())



In [ ]:
df

In [ ]:
# Pour sauvegarder :
df.to_csv("modules_units_content.csv", index=False)